# Image Cleaner

In [1]:
# import constants and helpers
%run Constants.ipynb

In [2]:
# how the validation dataframe has been created:
import csv
from pathlib import Path
import geopandas as gpd
import json
# columns:
# image_name, is_crossing, crossing_location, invalid_image
fieldnames = [
    "image_name",
    "is_crossing",
    "crossing_location",  # json string
    "invalid_image",
]

def get_validated_images():
    column_conversion = {
        "image_name": str,
        "is_crossing": bool,
        "crossing_location": lambda x: json.loads(x) if x else None,
        "invalid_image": bool,
    }
    if not CLEANED_IMAGES_CSV.exists():
        return []
    with open(CLEANED_IMAGES_CSV, newline='') as csvfile:
        images = csv.DictReader(csvfile)
        results = []
        for row in images:
            row_result = {
                column_name : column_conversion[column_name](row[column_name]) for column_name in row
            }
            results.append(row_result)
    return results

def add_validated_image(*, image_path: Path, is_crossing: bool, crossing_location: str,  invalid_image: bool):
    insert_dict = {
        "image_name": image_path.name,
        "is_crossing": is_crossing,
        "crossing_location": json.dumps(crossing_location),
        "invalid_image": invalid_image,
    }
    if not CLEANED_IMAGES_CSV.exists():
        with open(CLEANED_IMAGES_CSV, newline='', mode='a') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()

    with open(CLEANED_IMAGES_CSV, newline='', mode='a') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writerow(insert_dict)

In [3]:
# uncomment to have a simple "test-case"
# add_validated_image(
#     image_path=Path('/home/jovyan/work/OSMDeepOD/src/pre-processing-data/images/source/crossings/8.849166870117188_47.75629425048828_is_crossing_1.tif'),
#     is_crossing=True,
#     crossing_location='{"some": "json"}',
#     invalid_image=False,
# )
# result = get_validated_images()
# [r for r in result if r['invalid_image'] == True][0]

In [4]:
from sidecar import Sidecar
import ipywidgets as widgets
from ipyleaflet import (
    Map, ImageOverlay, DrawControl, WidgetControl
)
from matplotlib import pyplot as plt

def get_draw_control():
    draw_control = DrawControl()
    draw_control.polyline =  {}
    draw_control.polygon = {
        "allowIntersection": False
    }
    draw_control.circle = {}
    draw_control.rectangle = {}
    draw_control.circlemarker = {}
    return draw_control


def create_map(*args, **kwargs) -> Map:
    default = dict(
        zoom=19,
        # zoom_control=True,
        # min_zoom=18,
        max_zoom=25,
        # dragging=False,
    )
    default.update(kwargs)
    m = Map(**default)
    display(m)
    sidecar = None
    # sidecar would only be needed to display it on the side of the notebook
    # but we're using voila, therefore it isn't used.
    # sidecar = Sidecar(title='Map')
    # with sidecar:
    #     display(m)
    draw_control = get_draw_control()
    m.add_control(draw_control)
    
    return m, sidecar, draw_control

In [5]:
import json

import rasterio as rio
from rasterio.plot import reshape_as_raster, reshape_as_image

def load_image(img_src, *, bands=[3,1,2], input_crs=CRS_2056, output_crs=CRS_4326, reduction_factor=6000):
    with rio.open(img_src, 'r') as src:
        bounds = shapely_to_geo_series_crs_conversion(shape=box(*src.bounds), input_crs=input_crs, output_crs=output_crs)
        center = [bounds.centroid.y,bounds.centroid.x]
        image_data = src.read([3,1,2])
    reduced = reduce_depth(image_data, display_min=0, display_max=reduction_factor)
    return reduced, bounds, center
    
def show_next_image_on_map(*, m: Map, overlay):
    global crossing
    global image

    checked_images = [r['image_name'] for r in get_validated_images()]
    image = next(crossing)
    
    # skip already checked images
    while image.name in checked_images:
        image = next(crossing)

    if overlay:
        m.remove_layer(overlay)
    image_reduced, bounds, center = load_image(image)
    bounds = bounds.bounds
    image_bounds = [[bounds[3], bounds[2]],[bounds[1], bounds[0]]]
    map_image = reshape_as_image(image_reduced)
    im_overlay = make_imageoverlay(map_image, image_bounds)
    m.add_layer(im_overlay)
    m.zoom = 20
    m.center = center
    # m.fit_bounds(image_bounds)
    return im_overlay, image


def create_button_widgets(*, options: dict, position='topright'):
    buttons = []
    for name, content in options.items():
        button = widgets.Button(
            description=name,
            disabled=False,
            button_style=content.get('style', 'info'), # 'success', 'info', 'warning', 'danger' or ''
            tooltip=name,
            layout=widgets.Layout(width='200px', height='100%'),
            # icon='check' # (FontAwesome names without the `fa-` prefix)
        )
        button.on_click(content.get('callback'))
        buttons.append(button)
    hbox_widget = widgets.HBox(
        buttons,
        align_items='stretch',
        border='solid',
        width='900px',
    )
    widget_control = WidgetControl(widget=hbox_widget, position=position)
    return widget_control

def save_image_state(invalid_image=False, is_crossing=True, crossing_location=None):
    global im_overlay
    global draw_control
    global image
    global crossing
    global m

    add_validated_image(
        image_path=image,
        is_crossing=is_crossing,
        crossing_location=crossing_location,
        invalid_image=invalid_image,
    )
    m.remove_control(draw_control)
    draw_control = get_draw_control()
    m.add_control(draw_control)
    
    im_overlay, image = show_next_image_on_map(m=m, overlay=im_overlay)


def is_defective_image(*args, **kwargs):
    print(*args, **kwargs)
    save_image_state(invalid_image=True, is_crossing=None)


def no_crossing_on_image(*args, **kwargs):
    print(*args, **kwargs)
    save_image_state(invalid_image=False, is_crossing=False)


def set_bbox_on_crossing_image(*args, **kwargs):
    global draw_control
    # don't react, when trying to save a crossing without drawing
    crossing_json = draw_control.get_state().get('data')
    if not crossing_json:
        return
    print(*args, **kwargs)
    save_image_state(invalid_image=False, is_crossing=True, crossing_location=crossing_json)


buttons = {
    'defective': {
        "callback": is_defective_image,
        "style": "danger",
    },
    'no crossing': {
        "callback": no_crossing_on_image,
        "style": "info",
    },
    'save marked crossing': {
        "callback": set_bbox_on_crossing_image,
        "style": "success",
    },
}

In [6]:
RAPPPI_CENTER = [47.2245967,8.8168816]
m, sidecar, draw_control = create_map(center=RAPPPI_CENTER)

# ugly global state
im_overlay = None
crossing = IMAGES_TIF_FOLDER_CROSSINGS.glob("*.tif")

im_overlay, image = show_next_image_on_map(m=m, overlay=im_overlay)
button_control = create_button_widgets(options=buttons)
m.add_control(button_control)

Map(center=[47.2245967, 8.8168816], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title'…

In [ ]:
# im_overlay = show_next_image_on_map(next(crossing), m=m, overlay=im_overlay)
# draw_control.get_state().get('data')

In [ ]:
# todo: get data from draw_control
# draw_control.get_state().get('data')
# m.remove_layer(im_overlay)

In [ ]:
# sidecar.close()

# Appendix

## find a decent reduction value


In [ ]:
import geopandas as gpd
import rasterio as rio
from shapely.geometry import box
from rasterio.plot import reshape_as_raster, reshape_as_image

def show_example_image_with_factor(image, factor):
    EXAMPLE_IMAGE = IMAGES_TIF_FOLDER_CROSSINGS / "8.474699974060059_47.40547180175781_is_crossing_1.tif"
    EXAMPLE_IMAGE.exists()

    with rio.open(EXAMPLE_IMAGE, 'r') as src:
        bounds = shapely_to_geo_series_crs_conversion(shape=box(*src.bounds), input_crs=CRS_2056, output_crs=CRS_4326)
        wgs84_center = [bounds.centroid.y,bounds.centroid.x]
        image_data = src.read([3,1,2])
        reduced = reduce_depth(image_data, display_min=0, display_max=reduction_factor)
    rio.plot.show(reduced)

In [ ]:
# uncomment to run

# reduction_factor = 6000
# example_image = next(IMAGES_TIF_FOLDER_CROSSINGS.glob('*.tif'))
# show_example_image_with_factor(example_image, reduction_factor)

In [ ]:
# relicts: finding image border
# image_bounds = bounds.bounds
# image_bounds = [[image_bounds[3], image_bounds[2]],[image_bounds[1], image_bounds[0]]]
# print(wgs84_center)
# print(image_bounds)